## Reinforcement learning

## Installation

In [1]:
#@title Install `stable-baseline3` on Colab

!apt-get update && apt-get install swig cmake
!pip install box2d-py
!pip install gymnasium
!pip install "stable-baselines3[extra]>=2.0.0a4"

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 6.4 MB/s eta 0:00:00 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for box2d-py (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [16 lines of output]
      Using setuptools (version 67.7.2).
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-38
      creating build/lib.linux-x86_64-cpython-38/Box2D
      copying 

In [3]:
#@title Install `flygym` on Colab

# This block is modified from dm_control's tutorial notebook
# https://github.com/deepmind/dm_control/blob/main/tutorial.ipynb

# import subprocess
# if subprocess.run('nvidia-smi').returncode:
#     raise RuntimeError(
#         'Cannot communicate with GPU. '
#         'Make sure you are using a GPU Colab runtime. '
#         'Go to the Runtime menu and select Choose runtime type.')

print('Installing flygym')
!pip install -q --progress-bar=off 'flygym[mujoco] @ git+https://github.com/NeLy-EPFL/flygym.git'

# Configure dm_control to use the EGL rendering backend (requires GPU)
%env MUJOCO_GL=egl

print('Checking that the dm_control installation succeeded...')
try:
    from dm_control import suite
    env = suite.load('cartpole', 'swingup')
    pixels = env.physics.render()
except Exception as e:
    raise e from RuntimeError(
        'Something went wrong during dm_control installation. Check the shell '
        'output above for more information.\n'
        'If using a hosted Colab runtime, make sure you enable GPU acceleration '
        'by going to the Runtime menu and selecting "Choose runtime type".')
else:
    del pixels, suite

print('Checking that the flygym installation succeeded...')
try:
    import flygym
    from flygym import envs
except Exception as e:
    raise e from RuntimeError(
        'Something went wrong during flygym installation. Check the shell '
        'output above for more information.\n')
else:
    del envs, flygym

Installing flygym
env: MUJOCO_GL=egl
Checking that the dm_control installation succeeded...
Checking that the flygym installation succeeded...


## Demo: Cartpole

We will demonstrate the use of reinforcement learning in training a controller for Cartpole: a toy environment where you try to balance a vertical pole on a cart by moving the cart left and right.

Cartpole is a predefined Gym, which makes it very easy to initialize. In the following code, the `gym.make` function creates a Gym environment that has been registered to `Gym`; it is equivalent to our `nmf = NeuroMechFlyMuJoCo(...)` call. 

In [4]:
import gymnasium as gym

cartpole_env = gym.make('CartPole-v1', render_mode='rgb_array')

Next, we initialize a model using `stable-baselines3`. This, once again, is a one-liner:

In [5]:
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy

cartpole_model = PPO(MlpPolicy, cartpole_env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


We can now evaluate the untrained random policy for the task. Note that we wrap the environment around the `Monitor` class, which is used to keep track of information like episode reward.

In [6]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

mean_reward, std_reward = evaluate_policy(cartpole_model,
                                          Monitor(cartpole_env),
                                          n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:9.56 +/- 0.80


Remember the reward is simply the number of timesteps where the controller mangaged to keep the pole upright. The result is not very impressive.

Now, we can train the model for 10,000 iterations:

In [7]:
cartpole_model.learn(total_timesteps=10000, progress_bar=True)

Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.5     |
|    ep_rew_mean     | 21.5     |
| time/              |          |
|    fps             | 1034     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 24.6        |
|    ep_rew_mean          | 24.6        |
| time/                   |             |
|    fps                  | 864         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008606881 |
|    clip_fraction        | 0.0948      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000739   |
|    learning_rate        | 0.0003      |
|    loss                 | 5.45        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0142     |
|    value_loss           | 49.7        |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 32.8        |
|    ep_rew_mean          | 32.8        |
| time/                   |             |
|    fps                  | 825         |
|    iterations           | 3           |
|    time_elapsed         | 7           |
|    total_timesteps      | 6144        |
| train/                  |             |
|    approx_kl            | 0.009230178 |
|    clip_fraction        | 0.0574      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.666      |
|    explained_variance   | 0.131       |
|    learning_rate        | 0.0003      |
|    loss                 | 11.7        |
|    n_updates            | 20          |
|    policy_gradient_loss | -0.0136     |
|    value_loss           | 34          |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 42.8        |
|    ep_rew_mean          | 42.8        |
| time/                   |             |
|    fps                  | 811         |
|    iterations           | 4           |
|    time_elapsed         | 10          |
|    total_timesteps      | 8192        |
| train/                  |             |
|    approx_kl            | 0.008520385 |
|    clip_fraction        | 0.0943      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.636      |
|    explained_variance   | 0.203       |
|    learning_rate        | 0.0003      |
|    loss                 | 17.8        |
|    n_updates            | 30          |
|    policy_gradient_loss | -0.0227     |
|    value_loss           | 46.5        |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 57.5        |
|    ep_rew_mean          | 57.5        |
| time/                   |             |
|    fps                  | 792         |
|    iterations           | 5           |
|    time_elapsed         | 12          |
|    total_timesteps      | 10240       |
| train/                  |             |
|    approx_kl            | 0.008251837 |
|    clip_fraction        | 0.0843      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.61       |
|    explained_variance   | 0.262       |
|    learning_rate        | 0.0003      |
|    loss                 | 21.7        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.019      |
|    value_loss           | 62.9        |
-----------------------------------------


Let's reevaluate the model:

In [8]:
mean_reward, std_reward = evaluate_policy(cartpole_model,
                                          Monitor(cartpole_env),
                                          n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:386.01 +/- 105.28


Better. Let's take a look at a video of a simulation:

In [9]:
obs = cartpole_env.reset()
scenes = []
for i in range(500):
    action, _ = cartpole_model.predict(cartpole_env.state)
    obs, reward, terminated, truncated, info = cartpole_env.step(action)
    scenes.append(cartpole_env.render())
    if terminated:
        # stop early if the simulation terminates early because the pole fell
        break

In [10]:
import mediapy
mediapy.show_video(scenes)

This browser does not support the video tag.


## Controlling NeuroMechFly with RL

As discussed in the lecture, we will now try to control the stepping of each legs with reinforcement learning.

First we need to write our own Gym environment that functions as a "wrapper" around the underlying the `NeuroMechFlyMuJoCo` object. You can achieve this by implementing a class inheriting from `gym.Env` with the actual `NeuroMechFlyMuJoCo` simulation saved as an attribute.

There are three things to note here:
1. For the gym environment to work with models in Stable Baselines 3, the observation and action spaces have to be arrays instead of dictionaries of arrays. We do this by concatenating the flattened arrays into a single array.
2. Under `__init__`, you have to define the expected dimensions and bounds of the observation/action space, so the model knows what inputs/outputs are valid.
3. The `step` method has to return five values: the observation, the reward, whether the simulation is terminated, whether the simulation is truncated, and some additional info. This is different from `NeuroMechFlyMuJoCo`.

In [39]:
from gymnasium import spaces
from flygym.envs.nmf_mujoco import NeuroMechFlyMuJoCo
import numpy as np

class MyNMF(gym.Env):
    def __init__(self, **kwargs):
        self.nmf = NeuroMechFlyMuJoCo(**kwargs)
        num_dofs = len(self.nmf.actuated_joints)
        bound = 0.5
        self.action_space = spaces.Box(low=-bound, high=bound,
                                       shape=(num_dofs,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf,
                                            shape=(num_dofs,))
        self.joint_pos= np.zeros(num_dofs)
        self.fly_pos = np.zeros(3)
        self.fly_vel = np.zeros(3)

    
    def _parse_obs(self, raw_obs):
        features = [
            #joint angle
            raw_obs['joints'][0, :].flatten(),
            #joint velocity
            #raw_obs['joints'][1, :].flatten(),
            #joint torque
            #raw_obs['joints'][1, :].flatten(),
            # raw_obs['fly'].flatten(),
            # what else would you like to include?
        ]
        #print(raw_obs['joints'].shape)
        #print(np.array(features).shape)
        return np.concatenate(features, dtype=np.float32)
    
    def reset(self):
        raw_obs, info = self.nmf.reset()
        return self._parse_obs(raw_obs), info
    
    def reward(self):
        """ Reward function aiming at maximizing forward velocity."""
        # reward for forward velocity
        vel_tracking_reward = self.fly_vel[0]
        reward=vel_tracking_reward
        print(reward)

        #vel_tracking_reward = 0.2 * np.exp( -1/ 0.25 *  (self.robot.GetBaseLinearVelocity()[0] - des_vel_x)**2 )
        
        # # minimize yaw (go straight) 
        # yaw_reward = -0.3 * np.abs(self.robot.GetBaseOrientationRollPitchYaw()[2]) 

        # #minmize roll (not fall on the side)
        # roll_reward = -0.3 * np.abs(self.robot.GetBaseOrientationRollPitchYaw()[0]) 

        # # penalty body y - don't drift laterally
        # drift_reward = -0.01 * abs(self.robot.GetBasePosition()[1]) #0.01

        # #linear velocity penalty body z (mentioned in paper)
        # vel_body_z_penalty = -0.05*(self.robot.GetBaseLinearVelocity()[2])**2
        # # minimize energy 
        # energy_reward = 0 
        # for tau,vel in zip(self._dt_motor_torques,self._dt_motor_velocities):
        # energy_reward += np.abs(np.dot(tau,vel)) * self._time_step

        # reward = vel_tracking_reward \
        #         + yaw_reward \
        #         + roll_reward\
        #         + drift_reward \
        #         + vel_body_z_penalty \
        #         - 0.01 * energy_reward \
        #         - 0.1 * np.linalg.norm(self.robot.GetBaseOrientation() - np.array([0,0,0,1]))

        return max(reward,0) # keep rewards positive
        
    def step(self, action):
        raw_obs, info = self.nmf.step({'joints': action})
        obs = self._parse_obs(raw_obs)
        self.joint_pos = raw_obs['joints'][0, :]
        self.fly_pos = raw_obs['fly'][0, :]
        self.fly_vel = raw_obs['fly'][1, :]
        reward = self.reward()  # what is your reward function?
        terminated = False
        truncated = False
        return obs, reward, terminated, truncated, info

    def render(self):
        return self.nmf.render()
    
    def close(self):
        return self.nmf.close()

We can now train a agent on this environment:

In [40]:
from flygym.util.config import all_leg_dofs
from flygym.util.config import leg_dofs_3_per_leg

#using the config 3 Dofs per leg we just get the joint positions in observation
# if we would like to use other info in observation need to change observation space

run_time = 0.5
nmf_env_headless = MyNMF(render_mode='headless',
                         timestep=1e-4,
                         init_pose='stretch',
                         actuated_joints=leg_dofs_3_per_leg)  # which DoFs would you use?
nmf_model = PPO(MlpPolicy, nmf_env_headless, verbose=1)
nmf_model.learn(total_timesteps=100_000, progress_bar=True)
nmf_model.close()


Output()

-1225.5263505487558

-2323.9176790168763

-3379.88607422242

-4489.081675645516

...

-15560.490606822465

-15328.096085071009

-15608.028164462128

-15244.543197201874

-15213.262868144257

... and evaluate it:

In [ ]:
nmf_env_rendered = MyNMF(render_mode='saved',
                         timestep=1e-4,
                         init_pose='stretch',
                         render_config={'playspeed': 0.1,
                                        'camera': 'Animat/camera_left_top'},
                         actuated_joints=...)
obs, _ = nmf_env_rendered.reset()
obs_list = []
rew_list = []
for i in range(int(run_time / nmf_env_rendered.nmf.timestep)):
    action, _ = nmf_model.predict(obs)
    obs, reward, terminated, truncated, info = nmf_env_rendered.step(action)
    obs_list.append(obs)
    rew_list.append(reward)
    nmf_env_rendered.render()

We can also visualize the results:

In [ ]:
nmf_env_rendered.nmf.save_video('filename.mp4')
nmf_env_rendered.close()